# 6. Visualizing Ciona scRNA-Seq expression

This notebook illustrates a variety of functions for visualizing Ciona scRNA-Seq expression data across developmental stages, including:
- Violin plot distributions of gene expression across cell clusters identified in Seurat
- `[TBI]` UMAP scatter plots of gene expression, tissue type, and cluster identity
- `[TBI]` Tissue-specific bubble plots of gene expression across developmental stages

*`[TBI]` indicates that the functions will be implemented in a future PR.

To run this notebook, you should have run the downloading Snakemake workflow described in the README at the top level of this repo, followed by the Jupyter notebooks numbered 0, 1, 3, 4, and 5.

## 6.1 Load necessary modules

Be sure to install `zoogletools` from the top level of this directory.

In [1]:
from pathlib import Path

import numpy as np

import zoogletools as zt
from zoogletools.ciona.constants import (
    CAO_DATA_DIRPATH,
    CIONA_GENE_MODELS_DIRPATH,
    PIEKARZ_DATA_DIRPATH,
    ZOOGLE_RESULTS_DIRPATH,
    CionaStage,
)
from zoogletools.ciona.identifier_mapping import CionaIDTypes

/Users/dennis/miniconda3/envs/2025-zoogle/lib/python3.12/site-packages/anndata/utils.py:434: FutureWarning: Importing read_csv from `anndata` is deprecated. Import anndata.io.read_csv instead.
  warnings.warn(msg, FutureWarning)
/Users/dennis/miniconda3/envs/2025-zoogle/lib/python3.12/site-packages/anndata/utils.py:434: FutureWarning: Importing read_text from `anndata` is deprecated. Import anndata.io.read_text instead.
  warnings.warn(msg, FutureWarning)
/Users/dennis/miniconda3/envs/2025-zoogle/lib/python3.12/site-packages/anndata/utils.py:434: FutureWarning: Importing read_excel from `anndata` is deprecated. Import anndata.io.read_excel instead.
  warnings.warn(msg, FutureWarning)
/Users/dennis/miniconda3/envs/2025-zoogle/lib/python3.12/site-packages/anndata/utils.py:434: FutureWarning: Importing read_mtx from `anndata` is deprecated. Import anndata.io.read_mtx instead.
  warnings.warn(msg, FutureWarning)
/Users/dennis/miniconda3/envs/2025-zoogle/lib/python3.12/site-packages/anndata

## 6.2 Create identifier mapper object

Because there are multiple different identifiers for the same gene, we need to create an identifier mapper object that can map between different identifier types.

We create this once and reuse it for all plots in this notebook.

### Inputting identifiers
Plots in this module can work with the following identifiers, specified via the `CionaIDTypes` `StrEnum` class in the `input_id_type` parameter (see downstream functions for more details). The supported IDs include:
- **HGNC Gene Symbol** (`CionaIDTypes.HGNC_GENE_SYMBOL`). This is the primary gene symbol for the human gene.
- **Ciona UniProt ID** (`CionaIDTypes.NONREF_PROTEIN`). This is the UniProt ID of the Ciona protein.
- **Ciona KY2021 ID** (`CionaIDTypes.KY_ID`). This is the gene identifier from the KY2021 genome.
- **Ciona KH2012 ID** (`CionaIDTypes.KH_ID`). This is the gene identifier from the KH2012 genome.

For the scRNA-Seq data, the input identifier will be mapped to its corresponding `KY_ID`, which is the identifier used for gene expression within the Piekarz dataset.

In [2]:
id_mapper = zt.ciona.identifier_mapping.IdentifierMapper(
    zoogle_results_dirpath=ZOOGLE_RESULTS_DIRPATH,
    ciona_gene_models_dirpath=CIONA_GENE_MODELS_DIRPATH,
)

## 6.3 Violin plots of gene expression

`zoogletools` includes functions to visualize expression of user-selected genes across cell clusters. The clusters can be colorized either by the Seurat cluster number, or by the top most-represented tissue type found in the cluster.

These plots contain three subplots:
- 1. **Cell count bar chart.** The top row shows a bar chart of the number of cells in each cluster.
- 2. **Percent expression bar chart.** The middle row shows a bar chart where the proportion of cells with nonzero expression of the gene of interest is colored.
- 3. **Expression violin chart.** The bottom row shows a violin chart of the distribution of gene expression for cells with nonzero expression.

In [3]:
input_id = "NAXE"
stage = CionaStage.INIG

cluster_plot = zt.ciona.plotting.plot_expression_violin(
    stage=stage,
    input_id=input_id,
    input_id_type=CionaIDTypes.HGNC_GENE_SYMBOL,
    data_dirpath=PIEKARZ_DATA_DIRPATH,
    mapper=id_mapper,
    color_mode="cluster",
)

cluster_plot.show()

tissue_plot = zt.ciona.plotting.plot_expression_violin(
    stage=stage,
    input_id=input_id,
    input_id_type=CionaIDTypes.HGNC_GENE_SYMBOL,
    data_dirpath=PIEKARZ_DATA_DIRPATH,
    mapper=id_mapper,
    color_mode="tissue",
)

tissue_plot.show()

We can generate violin plots for multiple developmental stages at once using the `plot_expression_violin_for_all_stages` function. The code below generates violin plots for genes identified as tractable for pilot experiments in *Ciona*, colored by cluster and by tissue type, and saves the plots to the `figures` directory.

In [4]:
selected_genes = ["NAXE", "FCHO1", "PGM3", "RBBP7", "NCKAP1L"]

output_dirpath = "figures"

for gene in selected_genes:
    zt.ciona.plotting.plot_expression_violin_for_all_stages(
        input_id=gene,
        input_id_type=CionaIDTypes.HGNC_GENE_SYMBOL,
        data_dirpath=PIEKARZ_DATA_DIRPATH,
        mapper=id_mapper,
        color_mode="cluster",
        output_dirpath=output_dirpath,
    )

    zt.ciona.plotting.plot_expression_violin_for_all_stages(
        input_id=gene,
        input_id_type=CionaIDTypes.HGNC_GENE_SYMBOL,
        data_dirpath=PIEKARZ_DATA_DIRPATH,
        mapper=id_mapper,
        color_mode="tissue",
        output_dirpath=output_dirpath,
    )

100%|██████████| 10/10 [00:10<00:00,  1.03s/it]


## 6.4 Generating UMAP scatter plots

The `zoogletools` package also contains functions for generating UMAP plots showing the expression of individual genes and their tissue context.

In [5]:
input_id = "NAXE"
stage = CionaStage.INIG

umap_plot = zt.ciona.plotting.plot_expression_umap(
    stage=stage,
    input_id=input_id,
    input_id_type=CionaIDTypes.HGNC_GENE_SYMBOL,
    mapper=id_mapper,
    data_dirpath=PIEKARZ_DATA_DIRPATH,
    annotation_data_dirpath=CAO_DATA_DIRPATH,
)

umap_plot.show()

In [6]:
selected_genes = ["NAXE", "FCHO1", "PGM3", "RBBP7", "NCKAP1L"]

output_dirpath = "figures"

for gene in selected_genes:
    zt.ciona.plotting.plot_expression_umap_for_all_stages(
        input_id=gene,
        input_id_type=CionaIDTypes.HGNC_GENE_SYMBOL,
        mapper=id_mapper,
        data_dirpath=PIEKARZ_DATA_DIRPATH,
        annotation_data_dirpath=CAO_DATA_DIRPATH,
        output_dirpath=output_dirpath,
    )

100%|██████████| 10/10 [00:33<00:00,  3.32s/it]


## 6.5 Bubble plots

For a more comprehensive view of gene expression across tissue types and developmental stages, we can also generate bubble plots of expression. In these plots, the size of the outer circle reflects the total number of cells of a given tissue type, and the size of the inner circle reflects the number of cells of that tissue type that express the selected gene. The intensity of color of the inner circle reflects the mean expression for expressing cells.

In [7]:
input_id = "NAXE"
input_id_type = CionaIDTypes.HGNC_GENE_SYMBOL

# First, collect gene expression information across developmental stages
# and tissue types.

data_collector = zt.ciona.plotting.get_tissue_expression_data(
    input_id=input_id,
    input_id_type=input_id_type,
    data_dirpath=PIEKARZ_DATA_DIRPATH,
    annotation_data_dirpath=CAO_DATA_DIRPATH,
)
display(data_collector)

# Then, generate the bubble plot.
zt.ciona.plotting.plot_expression_bubbles(
    data_collector,
    input_id,
)

,Tissue Type,mean_expression,n_expressing_cells,n_cells,stage,percent_expressing
0,endoderm,1.446382,11,172,iniG,6.395349
1,epidermis,1.183485,54,925,iniG,5.837838
2,germ,1.060140,4,52,iniG,7.692308
3,mesenchyme,0.952485,5,77,iniG,6.493506
4,muscle-heart,1.160953,6,146,iniG,4.109589
...,...,...,...,...,...,...
3,mesenchyme,1.990547,61,1718,larva,3.550640
4,muscle-heart,2.026616,5,189,larva,2.645503
5,nervous-system,1.797076,13,338,larva,3.846154
6,notochord,NaN,0,3,larva,0.000000


In the default version of the plot, the area of the circles is directly proportional to the number of cells. However, this can make it difficult to visualize the degree of gene expression for some tissue types with very small numbers of cells.

You can use the `size_scaling_function` parameter to adjust the method for scaling cell counts to circle diameter. By default, this is `lambda x: np.sqrt(x) * 1.1`. A scaling function of `lambda x: np.cbrt(x) * 4` produces a plot with more visible plots (although the proportional size of the expressing cells vs. total cells is no longer exact).

In [8]:
zt.ciona.plotting.plot_expression_bubbles(
    data_collector, input_id, size_scaling_function=(lambda x: np.cbrt(x) * 4)
)

To generate plots for multiple genes, we can follow the approach below.

In [10]:
selected_genes = ["NAXE", "FCHO1", "PGM3", "RBBP7", "NCKAP1L"]

output_dirpath = Path("figures")

for gene in selected_genes:
    mapper = zt.ciona.identifier_mapping.IdentifierMapper()
    all_ids = mapper.map_to_all(input_id=gene, input_type=CionaIDTypes.HGNC_GENE_SYMBOL)

    uniprot_id = all_ids[CionaIDTypes.NONREF_PROTEIN]

    data_collector = zt.ciona.plotting.get_tissue_expression_data(
        gene, CionaIDTypes.HGNC_GENE_SYMBOL
    )

    fig = zt.ciona.plotting.plot_expression_bubbles(
        data_collector,
        gene,
        image_filepath=(
            output_dirpath / f"{gene}_{uniprot_id}_expression" / f"{gene}_bubble_chart.svg"
        ),
        html_filepath=(
            output_dirpath / f"{gene}_{uniprot_id}_expression" / f"{gene}_bubble_chart.html"
        ),
    )